In [18]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import colorlover as cl
import pandas as pd
import re
import nltk
import string
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding, Dropout, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
df = pd.read_csv("books/train.csv")

In [176]:
dfx = df[df['title']=="King, the Mice and the Cheese (Beginner Books)"].copy()
dfx.shape

(16, 5)

In [177]:
stop_words = stopwords.words("english")
def text_preproc(x):
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = "".join(x.split("<br />"))
    x = x.encode('ascii', 'ignore').decode()
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'@\S+', ' ', x)
    x = re.sub(r'#\S+', ' ', x)
    x = re.sub(r'\'\w+', '', x)
#     x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x = re.sub(r'\w*\d+\w*', '', x)
    x = re.sub(r'\s{2,}', ' ', x)
    return x

In [178]:
model = Word2Vec.load("books/W2V.model")
modelX = tf.keras.models.load_model('model.h5')

In [179]:
corpus = ''
for text in dfx['review']:
    corpus += text

In [180]:
aspect = ['plot','character','setting','conflict','style','narration']
plot_score = []
char_score = []
setting_score = []
conflict_score = []
style_score = []
narr_score = []
# overall_score = []
voc_size = 10000
sent_length = 50

In [181]:
aspect_count = 0
aspect_name = ''
aspect_count1 = 0
aspect_name1 = ''
sentence_raw = nltk.sent_tokenize(corpus)
sentence = [text_preproc(sen) for sen in sentence_raw]
for sent in sentence:
#     print(sent, end="____________")
    words = nltk.word_tokenize(sent)
    pos = nltk.pos_tag(words)
    for (word,cat) in pos:
#         print(word)
        if (word in model.wv.vocab) == True and (cat == 'NN' or cat == 'NNP'):
#             print('yes')
            for asp in aspect:
                if model.wv.similarity(asp, word)<0.3:
                    continue
                if model.wv.similarity(asp, word) > aspect_count:
#                     print(model.wv.similarity(asp, word))
                    aspect_count1 = aspect_count
                    aspect_name1 = aspect_name
                    aspect_count = model.wv.similarity(asp, word)
                    aspect_name = asp
#                     print(aspect_count)
#     print((aspect_name,aspect_count),(aspect_name1,aspect_count1))
    onehot_reprx = [one_hot(sent,voc_size)]
    testing_x=pad_sequences(onehot_reprx,padding='pre',maxlen=sent_length)
    testing_x.shape
    score = modelX.predict(testing_x)[0][0]
    aspect_score = score
    aspect_score1 = score
    if(aspect_name=='plot'):
        plot_score.append(aspect_score)
    elif(aspect_name=='character'):
        char_score.append(aspect_score)
    elif(aspect_name=='setting'):
        setting_score.append(aspect_score)
    elif(aspect_name=='conflict'):
        conflict_score.append(aspect_score)
    elif(aspect_name=='style'):
        style_score.append(aspect_score)
    elif(aspect_name=='narration'):
        narr_score.append(aspect_score)
        
    if(aspect_name1=='plot'):
        plot_score.append(2*aspect_count1*score)
    elif(aspect_name1=='character'):
        char_score.append(2*aspect_count1*score)
    elif(aspect_name1=='setting'):
        setting_score.append(2*aspect_count1*score)
    elif(aspect_name1=='conflict'):
        conflict_score.append(2*aspect_count1*score)
    elif(aspect_name1=='style'):
        style_score.append(2*aspect_count1*score)
    elif(aspect_name1=='narration'):
        narr_score.append(2*aspect_count1*score)
    
    aspect_count = 0
    aspect_name = ''
    aspect_count1 = 0
    aspect_name1 = ''

In [182]:
onehot_reprx = []
# s = "i can't believe people are looking for a plot in this film"
s = sentence
onehot_reprx=onehot_reprx+[one_hot(sents,voc_size)for sents in s] 
testing_x=pad_sequences(onehot_reprx,padding='pre',maxlen=sent_length)
testing_x.shape

(55, 50)

In [186]:
aspect_df = aspect + ["overall"]
pos = [sum(plot_score)/len(plot_score), sum(char_score)/len(char_score), sum(setting_score)/len(setting_score), sum(conflict_score)/len(conflict_score), sum(style_score)/len(style_score), sum(narr_score)/len(narr_score), modelX.predict(testing_x)[0][0]]
neg = [sum(plot_score)/len(plot_score)-1, sum(char_score)/len(char_score)-1, sum(setting_score)/len(setting_score)-1, sum(conflict_score)/len(conflict_score)-1, sum(style_score)/len(style_score)-1, sum(narr_score)/len(narr_score)-1, modelX.predict(testing_x)[0][0]-1]

In [187]:
char_score

[]

In [188]:
plotly_df = pd.DataFrame(list(zip(aspect_df,neg,pos)),columns=['aspect','neg','pos'])
plotly_df.set_index('aspect', inplace=True)
plotly_df

,neg,pos
aspect,,
plot,-0.346981,0.653019
character,0.000000,0.000000
setting,-0.394716,0.605284
conflict,-0.654029,0.345971
style,-0.652122,0.347878
narration,-0.999941,0.000059
overall,-0.045158,0.954842


In [189]:
# sort by desired column
category_order = [
    'neg',
    'pos'
]
plotly_df = plotly_df.sort_values(by='neg', ascending = False)

fig = go.Figure()

for column in plotly_df.columns:
    fig.add_trace(go.Bar(
        x = plotly_df[column],
        y = plotly_df.index,
        name = column,
        orientation = 'h',
#         marker_color = cl.scales[str(len(category_order))]['div']['RdYlGn'][category_order.index(column)],
    ))

fig.update_layout(
    barmode = 'relative',
    title = 'Aspect Based Sentiment Score of Books'
)
fig.show()

In [190]:
sim = model.wv.most_similar('conflict', topn=50)
sim

[('conflicts', 0.8543586730957031),
 ('strife', 0.7257596850395203),
 ('tensions', 0.6902616024017334),
 ('turmoil', 0.6445066928863525),
 ('crises', 0.6368471384048462),
 ('rebellion', 0.6221489310264587),
 ('misunderstanding', 0.6198573112487793),
 ('unrest', 0.6192880868911743),
 ('confrontation', 0.6164884567260742),
 ('crisis', 0.61197829246521),
 ('tension', 0.5999663472175598),
 ('confusion', 0.5924843549728394),
 ('catastrophe', 0.5909031629562378),
 ('uncertainty', 0.58596271276474),
 ('rivalry', 0.5786142349243164),
 ('upheaval', 0.5750538110733032),
 ('threat', 0.5719186067581177),
 ('ramifications', 0.5687743425369263),
 ('battles', 0.5685055255889893),
 ('ambiguity', 0.5684546232223511),
 ('bloodshed', 0.5646613836288452),
 ('aggression', 0.5633753538131714),
 ('complications', 0.5620979070663452),
 ('battle', 0.5611540079116821),
 ('animosity', 0.5594679713249207),
 ('rift', 0.5494676828384399),
 ('misunderstandings', 0.5477968454360962),
 ('drama', 0.5450290441513062),
 